In [3]:
!pip install "mlflow<2.14.0"

  Using cached packaging-24.2-py3-none-any.whl.metadata (3.2 kB)
   ---------------------------------------- 0.0/25.0 MB ? eta -:--:--
   --- ------------------------------------ 2.1/25.0 MB 114.7 MB/s eta 0:00:01
   ------ --------------------------------- 4.2/25.0 MB 12.6 MB/s eta 0:00:02
   -------- ------------------------------- 5.2/25.0 MB 8.4 MB/s eta 0:00:03
   -------------------- ------------------- 12.8/25.0 MB 15.5 MB/s eta 0:00:01
   ----------------------- ---------------- 14.7/25.0 MB 17.7 MB/s eta 0:00:01
   ----------------------------------- ---- 22.0/25.0 MB 19.3 MB/s eta 0:00:01
   ---------------------------------------- 25.0/25.0 MB 17.8 MB/s  0:00:01
   ---------------------------------------- 0.0/24.8 MB ? eta -:--:--
   ---------------- ----------------------- 10.5/24.8 MB 50.4 MB/s eta 0:00:01
   ----------------------- ---------------- 14.7/24.8 MB 35.5 MB/s eta 0:00:01
   ---------------------------- ----------- 17.6/24.8 MB 28.4 MB/s eta 0:00:01
   --------

  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opentelemetry-proto 1.39.1 requires protobuf<7.0,>=5.0, but you have protobuf 4.25.8 which is incompatible.


In [1]:
import pandas as pd
from pycaret.classification import *

# Load the clean dataset
df = pd.read_csv('heart.csv')

# Handle the 0 Cholesterol issue just like in Task 1
df['Cholesterol'] = df['Cholesterol'].replace(0, pd.NA)
df['Cholesterol'] = df['Cholesterol'].fillna(df['Cholesterol'].median())
df = df[df['RestingBP'] > 0]

# Sample 5% of the data to act as completely unseen data
data_unseen = df.sample(frac=0.05, random_state=42)
data_train = df.drop(data_unseen.index)

print(f"Data for Modeling: {data_train.shape}")
print(f"Unseen Data For Predictions: {data_unseen.shape}")

Data for Modeling: (871, 12)
Unseen Data For Predictions: (46, 12)


In [2]:
# This satisfies the scaling, transformation, and MLFlow requirements
experiment = setup(data=data_train, 
                   target='HeartDisease', 
                   session_id=123, 
                   normalize=True, 
                   transformation=True, 
                   log_experiment=True, 
                   experiment_name='heart_failure_mlops')

,Description,Value
0,Session id,123
1,Target,HeartDisease
2,Target type,Binary
3,Original data shape,"(871, 12)"
4,Transformed data shape,"(871, 19)"
5,Transformed train set shape,"(609, 19)"
6,Transformed test set shape,"(262, 19)"
7,Numeric features,6
8,Categorical features,5
9,Preprocess,True


2026/02/20 17:40:16 INFO mlflow.tracking.fluent: Experiment with name 'heart_failure_mlops' does not exist. Creating a new experiment.


In [3]:
# Train and validate models using k-fold cross-validation
best_model = compare_models()

# Tune the best model
tuned_model = tune_model(best_model)

# Evaluate the model (This opens the interactive dashboard)
evaluate_model(tuned_model)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
rf,Random Forest Classifier,0.8687,0.9083,0.9193,0.8571,0.8852,0.7323,0.7396,0.0280
lightgbm,Light Gradient Boosting Machine,0.8555,0.9092,0.8863,0.8578,0.8709,0.7069,0.7093,0.0710
ada,Ada Boost Classifier,0.8523,0.9076,0.8714,0.8648,0.8662,0.7013,0.7051,0.0160
ridge,Ridge Classifier,0.8457,0.9114,0.8715,0.8538,0.8607,0.6876,0.6914,0.0100
lda,Linear Discriminant Analysis,0.8440,0.9114,0.8684,0.8532,0.8591,0.6843,0.6875,0.0100
gbc,Gradient Boosting Classifier,0.8424,0.9069,0.8744,0.8483,0.8594,0.6799,0.6837,0.0200
lr,Logistic Regression,0.8407,0.9091,0.8654,0.8509,0.8563,0.6776,0.6812,0.2640
et,Extra Trees Classifier,0.8407,0.9021,0.8743,0.8460,0.8580,0.6766,0.6814,0.0250
knn,K Neighbors Classifier,0.8358,0.8832,0.8592,0.8471,0.8517,0.6676,0.6705,0.2140
nb,Naive Bayes,0.8277,0.9008,0.8472,0.8435,0.8437,0.6514,0.6544,0.1680


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8852,0.9361,0.9394,0.8611,0.8986,0.7670,0.7709
1,0.8197,0.8074,0.9394,0.7750,0.8493,0.6299,0.6481
2,0.9016,0.9307,0.9091,0.9091,0.9091,0.8019,0.8019
3,0.8852,0.9102,0.8485,0.9333,0.8889,0.7708,0.7745
4,0.8689,0.9253,0.9091,0.8571,0.8824,0.7345,0.7361
5,0.7869,0.9227,0.8235,0.8000,0.8116,0.5664,0.5667
6,0.8852,0.9314,0.9412,0.8649,0.9014,0.7647,0.7687
7,0.9180,0.9455,0.9118,0.9394,0.9254,0.8345,0.8350
8,0.8197,0.8878,0.8824,0.8108,0.8451,0.6303,0.6335


Fitting 10 folds for each of 10 candidates, totalling 100 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

In [4]:
# Predict on the 5% unseen data we held back earlier
unseen_predictions = predict_model(tuned_model, data=data_unseen)
print(unseen_predictions.head())

# Finalize the model (trains on the entire data_train set)
final_model = finalize_model(tuned_model)

# Save the entire pipeline (This generates the .pkl file for Task 3)
save_model(final_model, 'heart_failure_pipeline')

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Random Forest Classifier,0.8696,0.9229,0.9310,0.8710,0.9000,0.7131,0.7164


     Age Sex ChestPainType  RestingBP  Cholesterol  FastingBS RestingECG  \
669   45   F           ATA        130        234.0          0        LVH   
30    53   M           NAP        145        518.0          0     Normal   
377   65   M           ASY        160        237.0          1         ST   
536   62   M           NAP        133        237.0          1         ST   
807   54   M           ATA        108        309.0          0     Normal   

     MaxHR ExerciseAngina  Oldpeak ST_Slope  HeartDisease  prediction_label  \
669    175              N      0.6     Flat             0                 0   
30     130              N      0.0     Flat             1                 1   
377    122              N      1.2     Flat             1                 1   
536    119              Y      1.2     Flat             1                 1   
807    156              N      0.0       Up             0                 0   

     prediction_score  
669              0.66  
30               0.7

(Pipeline(memory=Memory(location=None),
          steps=[('numerical_imputer',
                  TransformerWrapper(exclude=None,
                                     include=['Age', 'RestingBP', 'Cholesterol',
                                              'FastingBS', 'MaxHR', 'Oldpeak'],
                                     transformer=SimpleImputer(add_indicator=False,
                                                               copy=True,
                                                               fill_value=None,
                                                               keep_empty_features=False,
                                                               missing_values=nan,
                                                               strategy='mean'))),
                 ('categorical_imputer',
                  TransformerWrapper(ex...
                  RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                         class_weight=None,